# *Shows sliders of the normal dosage: then adjust what is not normal and hit submit; yields data to be added to archieves*
1. **blnk_update_dt_row_dict**
2. **get differnt dates by computing it** *done in main_workbook*
3. *get using csv_file* +**load_lut_config(csv_path)**
4. *configure8 **slider properties with the data** *[Change by loading in XL]*

In [1]:
import ipywidgets as widgets
from IPython.display import display
import pickle

# Output container
output = widgets.Output()

# Input widgets
start_dtv = widgets.IntText(description='Start DTV:')
end_dtv = widgets.IntText(description='End DTV:')
submit_button = widgets.Button(description='Submit DTV')

# Callback
def on_submit(b):
    with output:
        output.clear_output()
        dtv = [start_dtv.value, end_dtv.value]
        print("DTV range:", dtv)
        with open("dtv_input.pkl", "wb") as f:
            pickle.dump(dtv, f)

# Bind and display
submit_button.on_click(on_submit)
display(start_dtv, end_dtv, submit_button, output)


IntText(value=0, description='Start DTV:')

IntText(value=0, description='End DTV:')

Button(description='Submit DTV', style=ButtonStyle())

Output()

In [21]:
import pickle
with open("dtv_input.pkl", "rb") as f:
    dtv = pickle.load(f)
print("Loaded DTV range:", dtv)


Loaded DTV range: [45862, 45863]


In [22]:
dtv

[45862, 45863]

In [23]:
import ipywidgets as widgets
from IPython.display import display
import pickle

In [24]:
# Load blank update dictionary
with open("blnk_update_dt_row_dict.pkl", "rb") as f:
    blnk_update_dt_row_dict = pickle.load(f)
blnk_dt_row_dict = blnk_update_dt_row_dict

In [25]:
blnk_dt_row_dict

{'45861': {'s000': '', 's001': '', 's002': '', 's003': '', 's004': ''},
 '45862': {'s000': '', 's001': '', 's002': '', 's003': '', 's004': ''},
 '45863': {'s000': '', 's001': '', 's002': '', 's003': '', 's004': ''},
 '45864': {'s000': '', 's001': '', 's002': '', 's003': '', 's004': ''},
 '45865': {'s000': '', 's001': '', 's002': '', 's003': '', 's004': ''}}

In [28]:
# def select_dtv_rows(dtv_range, blnk_dt_row_dict):
def select_dtv_rows(dtv_range, source_dict):
    # Ensure keys are sorted numerically
    sorted_keys = sorted(source_dict.keys(), key=int)

    # Unpack and normalize range
    start, end = sorted(dtv_range)
    
    # Filter keys within range
    selected = {
        k: source_dict[k]
        for k in sorted_keys
        if start <= int(k) <= end
    }
    return selected

# Example usage
# dtv = [45962, 45864]  Set ealier
blnk_update_dt_row_dict = select_dtv_rows(dtv, blnk_dt_row_dict)

#selected_rows = select_dtv_rows(dtv, blnk_my_dt_row_dict)
#print(selected_rows)


In [29]:
blnk_update_dt_row_dict 

{'45862': {'s000': '', 's001': '', 's002': '', 's003': '', 's004': ''},
 '45863': {'s000': '', 's001': '', 's002': '', 's003': '', 's004': ''}}

In [30]:
# 🔹 Step 1: Load Inputs
import pickle
import csv

# Load blank update dictionary
#with open("blnk_update_dt_row_dict.pkl", "rb") as f:
#    blnk_update_dt_row_dict = pickle.load(f)


# Load and parse lut_serial_config.csv
def load_lut_config(csv_path):
    with open(csv_path, newline='') as f:
        reader = list(csv.reader(f))
    headers = reader[0][1:]  # ['s000', 's001', ...]
    lut = {}
    
    for row in reader[1:]:
        key = row[0].strip('"')
        for i, val in enumerate(row[1:]):
            serial = headers[i]
            lut.setdefault(serial, {})[key] = val.strip('"')
    # Convert types
    for serial, config in lut.items():
        config['value'] = int(config.get('value', 1))
        config['min'] = int(config.get('min', 0))
        config['max'] = int(config.get('max', 10))
        config['step'] = int(config.get('step', 1)) if config.get('step') else 1
        config['active'] = config.get('active', 'true').lower() in ['true', 'yes']
    return lut

lut_serial_config = load_lut_config("lut_serial_config.csv")


In [31]:
blnk_update_dt_row_dict

{'45862': {'s000': '', 's001': '', 's002': '', 's003': '', 's004': ''},
 '45863': {'s000': '', 's001': '', 's002': '', 's003': '', 's004': ''}}

In [32]:
#lut_serial_config

In [33]:
# Step 2: Build Slider Form
import ipywidgets as widgets
from IPython.display import display, clear_output

update_dt_row_dict = {day: dict(serials) for day, serials in blnk_update_dt_row_dict.items()}
output = widgets.Output()
display(output)

form_items = []
slider_widgets = {}

for day, serials in update_dt_row_dict.items():
    form_items.append(widgets.HTML(value=f"<b>Day: {day}</b>"))
    for serial in serials:
        config = lut_serial_config.get(serial, {})
        if not config.get('active', True):
            continue
        slider = widgets.FloatSlider(
            value=config['value'],
            min=config['min'],
            max=config['max'],
            step=config['step'],
            description=f"{config['name']}:",
            layout=widgets.Layout(width='400px')
        )
        slider_widgets[(day, serial)] = slider
        form_items.append(slider)

submit_button = widgets.Button(description="Submit")

def on_submit(b):
    for (day, serial), slider in slider_widgets.items():
        update_dt_row_dict[day][serial] = slider.value
    with open("update_result.pkl", "wb") as f:
        pickle.dump(update_dt_row_dict, f)
    with output:
        clear_output()
        print("✅ Update complete. Result saved to [[update_result.pkl]]")
        for day, serials in update_dt_row_dict.items():
            print(f"{day}: {serials}")

submit_button.on_click(on_submit)
form_items.append(submit_button)
form = widgets.VBox(form_items)

with output:
    clear_output()
    print("Adjust sliders for each active supplement. Suspended ones are skipped.\n")
    display(form)


Output()

In [34]:
update_dt_row_dict

{'45862': {'s000': 0.0, 's001': 1.0, 's002': 2.0, 's003': 1.0, 's004': 1.0},
 '45863': {'s000': 1.0, 's001': 1.0, 's002': 2.0, 's003': 1.0, 's004': 4.0}}